# Li-ion Battery Aging Datasets (NASA): Data conversion

In this notebook, we convert all `.mat` datafiles in the Li-ion Battery Aging Datasets to csv files.

Reference:
B. Saha and K. Goebel (2007). "Battery Data Set", NASA Prognostics Data Repository, NASA Ames Research Center, Moffett Field, CA

```
Author: Cedric Yu
Last modified: 20230105
```

The raw `.mat` data files all have the following data structure:

```
Data Structure:
cycle:	top level structure array containing the charge, discharge and impedance operations
	type: 	operation  type, can be charge, discharge or impedance
	ambient_temperature:	ambient temperature (degree C)
	time: 	the date and time of the start of the cycle, in MATLAB  date vector format
	data:	data structure containing the measurements
	   for charge the fields are:
		Voltage_measured: 	Battery terminal voltage (Volts)
		Current_measured:	Battery output current (Amps)
		Temperature_measured: 	Battery temperature (degree C)
		Current_charge:		Current measured at charger (Amps)
		Voltage_charge:		Voltage measured at charger (Volts)
		Time:			Time vector for the cycle (secs)
	   for discharge the fields are:
		Voltage_measured: 	Battery terminal voltage (Volts)
		Current_measured:	Battery output current (Amps)
		Temperature_measured: 	Battery temperature (degree C)
		Current_charge:		Current measured at load (Amps)
		Voltage_charge:		Voltage measured at load (Volts)
		Time:			Time vector for the cycle (secs)
		Capacity:		Battery capacity (Ahr) for discharge till 2.7V 
	   for impedance the fields are:
		Sense_current:		Current in sense branch (Amps)
		Battery_current:	Current in battery branch (Amps)
		Current_ratio:		Ratio of the above currents 
		Battery_impedance:	Battery impedance (Ohms) computed from raw data
		Rectified_impedance:	Calibrated and smoothed battery impedance (Ohms) 
		Re:			Estimated electrolyte resistance (Ohms)
		Rct:			Estimated charge transfer resistance (Ohms)
```



In [43]:
# Load the "autoreload" extension so that code can change
import gc
import os
from utils.utils import load_mat_to_np, load_cycles_to_df, load_charge, load_discharge, load_impedance
%load_ext autoreload
# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
processed_dataset_folder = '../../li_ion_battery_aging_nasa/data/processed'
os.makedirs(processed_dataset_folder, exist_ok=True)


In [3]:
raw_dataset_folder = '../../li_ion_battery_aging_nasa/data/raw/'


In [8]:
sub_folders = os.listdir(raw_dataset_folder)
sub_folders


['1. BatteryAgingARC-FY08Q4',
 '2. BatteryAgingARC_25_26_27_28_P1',
 '4. BatteryAgingARC_45_46_47_48',
 '5. BatteryAgingARC_49_50_51_52',
 '3. BatteryAgingARC_25-44',
 '6. BatteryAgingARC_53_54_55_56']

In [9]:
os.listdir(os.path.join(raw_dataset_folder, sub_folders[0]))


['B0018.mat', 'B0007.mat', 'README.txt', 'B0006.mat', 'B0005.mat']

In [16]:
os.path.basename(os.listdir(os.path.join(
    raw_dataset_folder, sub_folders[0]))[0])


'B0018.mat'

In [12]:
for dirname, dirnames, filenames in os.walk(raw_dataset_folder):
    print(filenames)
    # # print path to all subdirectories first.
    # for subdirname in dirnames:
    #     print(os.path.join(dirname, subdirname))

    # # print path to all filenames.
    # for filename in filenames:
    #     print(filename)
    #     print(os.path.join(dirname, filename))


[]
['B0018.mat', 'B0007.mat', 'README.txt', 'B0006.mat', 'B0005.mat']
['B0025.mat', 'README.txt', 'B0027.mat', 'B0026.mat', 'B0028.mat']
['B0048.mat', 'B0045.mat', 'B0046.mat', 'README_45_46_47_48.txt', 'B0047.mat']
['README_49_50_51_52.txt', 'B0050.mat', 'B0049.mat', 'B0051.mat', 'B0052.mat']
['B0025.mat', 'B0039.mat', 'README_41_42_43_44.txt', 'README_33_34_36.txt', 'B0032.mat', 'B0031.mat', 'B0034.mat', 'README_38_39_40.txt', 'B0040.mat', 'B0041.mat', 'B0043.mat', 'README_29_30_31_32.txt', 'README_25_26_27_28.txt', 'B0036.mat', 'B0027.mat', 'B0044.mat', 'B0026.mat', 'B0042.mat', 'B0038.mat', 'B0028.mat', 'B0033.mat', 'B0029.mat', 'B0030.mat']
['B0056.mat', 'README_53_54_55_56.txt', 'B0053.mat', 'B0055.mat', 'B0054.mat']


In [48]:
error_files = []
for sub_folder in os.listdir(raw_dataset_folder):
    sub_folder_path = os.path.join(raw_dataset_folder, sub_folder)
    processed_dataset_sub_folder = os.path.join(
        processed_dataset_folder, sub_folder)
    os.makedirs(processed_dataset_sub_folder, exist_ok=True)
    mat_files = os.listdir(sub_folder_path)
    mat_files = [file for file in mat_files if file.split('.')[-1] == 'mat']

    for mat_file in mat_files:
        filename = mat_file.split('.')[0]
        filepath = os.path.join(sub_folder_path, mat_file)
        print(f'Processing {filepath}')
        try:
            df_charge_cycles, df_discharge_cycles, df_impedance_cycles = \
                load_cycles_to_df(load_mat_to_np(filepath))

            df_charge_cycles.to_csv(
                os.path.join(processed_dataset_sub_folder,
                             filename + '_charge.csv'),
                index=False)

            df_discharge_cycles.to_csv(
                os.path.join(processed_dataset_sub_folder,
                             filename + '_discharge.csv'),
                index=False)

            df_impedance_cycles.to_csv(
                os.path.join(processed_dataset_sub_folder,
                             filename + '_impedance.csv'),
                index=False)

        except Exception as e:
            print(e)
            error_files.append(filepath)

print(error_files)


Processing ../../li_ion_battery_aging_nasa/data/raw/1. BatteryAgingARC-FY08Q4/B0018.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/1. BatteryAgingARC-FY08Q4/B0007.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/1. BatteryAgingARC-FY08Q4/B0006.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/1. BatteryAgingARC-FY08Q4/B0005.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/2. BatteryAgingARC_25_26_27_28_P1/B0025.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/2. BatteryAgingARC_25_26_27_28_P1/B0027.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/2. BatteryAgingARC_25_26_27_28_P1/B0026.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/2. BatteryAgingARC_25_26_27_28_P1/B0028.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/4. BatteryAgingARC_45_46_47_48/B0048.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/4. BatteryAgingARC_45_46_47_48/B0045.mat
Processing ../../li_ion_battery_aging_nasa/data/raw/4. BatteryAgingARC_45_46_4

In [19]:
print(error_files)


['../../li_ion_battery_aging_nasa/data/raw/5. BatteryAgingARC_49_50_51_52/B0050.mat', '../../li_ion_battery_aging_nasa/data/raw/5. BatteryAgingARC_49_50_51_52/B0052.mat']
